In [1]:
from google.colab import drive
drive.mount('/mydrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /mydrive


In [7]:
cd /mydrive/My Drive/Colab Notebooks/Speech-Commands-Recognition

[Errno 2] No such file or directory: './mydrive/My Drive/Colab Notebooks/Speech-Commands-Recognition'
/


In [0]:
import librosa
from utils import util
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from tqdm import tqdm
from scipy import signal
import torch
from torchvision import datasets, transforms, models

In [0]:
def augentationMethod(data,dest_dir,curr_file,disr):
  sr = 16000
  ran = np.random.normal(0,2)
  if (ran >-0.8 and ran <0.8):
    return False
  elif (ran <=-0.8 ):# Time shifting method
      start_ = int(np.random.uniform(-4800,4800))
      if start_ >= 0:
        data = np.r_[data[start_:], np.random.uniform(-0.001,0.001, start_)]
      else:
        data = np.r_[np.random.uniform(-0.001,0.001, -start_), data[:start_]] 
        
  elif (ran >= 0.8 ):# Speed tuning
    speed_rate = np.random.uniform(0.7,1.3)
    wav_speed_tune = cv2.resize(data, (1, int(len(data) * speed_rate))).squeeze()
    if len(wav_speed_tune) < 16000:
      pad_len = 16000 - len(wav_speed_tune)
      data = np.r_[np.random.uniform(-0.001,0.001,int(pad_len/2)),
                           wav_speed_tune,
                           np.random.uniform(-0.001,0.001,int(np.ceil(pad_len/2)))]
    else: 
      cut_len = len(wav_speed_tune) - 16000
      data = wav_speed_tune[int(cut_len/2):int(cut_len/2)+16000]
  freq, times, spec = util.log_spectogram(data, sr)
  fig, ax = plt.subplots(1)
  fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
  ax.axis('off')
  ax.imshow(spec.T, aspect='auto', origin='lower')
  ax.axis('off')
  fig.savefig(f'{dest_dir}/{disr}/{curr_file[:-4]}'+str(ran)+'.jpg')
  plt.close(fig)

In [0]:
import traceback

In [0]:
def augment_audio(src_dir, dest_dir):
    """
    Generate spectograms from audio files.

    Parameters:
        src_dir - directory containing folders of different classes 
                  which contain their respective audio files
        dest_dir - directory to store spectograms
                   (directory should contain empty folders of respective classes)

    """
    # src_dir = '../datasets/data_audio/train/'
    # dest_dir = '../datasets/data_images/train/'
    dirs = ['yes','on','no','off', 'up', 'down', 'left',
            'right', 'go', 'stop'] 
    for dir in dirs:
        
          print(dir)
          audio_files = os.listdir(f'{src_dir}{dir}')
          j=0
          for curr_file in audio_files:
            try:
              j +=1
              data, sr = librosa.load(f'{src_dir}{dir}/{curr_file}', sr=16000)
              freq, times, spec = util.log_spectogram(data, sr)
              fig, ax = plt.subplots(1)
              fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
              ax.axis('off')
              ax.imshow(spec.T, aspect='auto', origin='lower')
              ax.axis('off')
              fig.savefig(f'{dest_dir}/{dir}/{curr_file[:-4]}.jpg')
              plt.close(fig)
              augentationMethod(data,dest_dir,curr_file,dir)
              if(j  == 1000):
                print('done 1000' + str(j) ,end='\r')
            except Exception:
              print('error')  
          print(j)

In [0]:
augment_audio("./PrepareData/dataset/","./PrepareData/dataset/train")

off
